In [3]:
import pandas as pd

In [4]:
file_name = 'FAOSTAT_data_11-18-2019.csv'

In [6]:
data = pd.read_csv(file_name)

In [21]:
data.head()

,Domain Code,Domain,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,TM,Detailed trade matrix,2,Afghanistan,100,India,5910,Export Quantity,515,Apples,2017,2017,tonnes,2092,R,Estimated data using trading partners database
1,TM,Detailed trade matrix,2,Afghanistan,100,India,5922,Export Value,515,Apples,2017,2017,1000 US$,2195,R,Estimated data using trading partners database
2,TM,Detailed trade matrix,2,Afghanistan,165,Pakistan,5910,Export Quantity,515,Apples,2017,2017,tonnes,74705,R,Estimated data using trading partners database
3,TM,Detailed trade matrix,2,Afghanistan,165,Pakistan,5922,Export Value,515,Apples,2017,2017,1000 US$,31501,R,Estimated data using trading partners database
4,TM,Detailed trade matrix,1,Armenia,73,Georgia,5910,Export Quantity,515,Apples,2017,2017,tonnes,39,NaN,Official data


In [25]:
only_quantity_data = data.loc[data['Element'] == 'Export Quantity']

In [26]:
only_quantity_data.head()

,Domain Code,Domain,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,TM,Detailed trade matrix,2,Afghanistan,100,India,5910,Export Quantity,515,Apples,2017,2017,tonnes,2092,R,Estimated data using trading partners database
2,TM,Detailed trade matrix,2,Afghanistan,165,Pakistan,5910,Export Quantity,515,Apples,2017,2017,tonnes,74705,R,Estimated data using trading partners database
4,TM,Detailed trade matrix,1,Armenia,73,Georgia,5910,Export Quantity,515,Apples,2017,2017,tonnes,39,NaN,Official data
6,TM,Detailed trade matrix,1,Armenia,225,United Arab Emirates,5910,Export Quantity,515,Apples,2017,2017,tonnes,0,NaN,Official data
8,TM,Detailed trade matrix,52,Azerbaijan,73,Georgia,5910,Export Quantity,515,Apples,2017,2017,tonnes,153,R,Estimated data using trading partners database


In [65]:
result = only_quantity_data.loc[(only_quantity_data['Reporter Country Code']==2) & (only_quantity_data['Partner Country Code']==10)]['Value']

In [67]:
# result.to_list()[0]

In [29]:
# unordered data structure of Python is set
# {1,2} == {2,1} # use set as key? it will be an undirected graph
# but what I need is a directed graph (A,B)= 515 means country A exports 515 tons apples to counry B

True

In [9]:
data.columns

Index(['Domain Code', 'Domain', 'Reporter Country Code', 'Reporter Countries',
       'Partner Country Code', 'Partner Countries', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value', 'Flag',
       'Flag Description'],
      dtype='object')

In [12]:
data.index

RangeIndex(start=0, stop=428, step=1)

In [30]:
start = data.groupby('Reporter Country Code').nunique().index

In [40]:
start_set = set(start)
print(len(start_set))

28


In [31]:
end = data.groupby('Partner Country Code').nunique().index

In [41]:
end_set = set(end)
print(len(end_set))

47


In [36]:
all_countries = start_set.union(end_set)

In [69]:
len(all_countries)
print(all_countries)

{1, 2, 13, 16, 26, 28, 38, 41, 299, 50, 52, 73, 96, 100, 101, 102, 103, 105, 108, 110, 112, 113, 115, 116, 117, 118, 120, 121, 128, 131, 132, 141, 149, 165, 171, 176, 179, 194, 200, 208, 212, 213, 214, 216, 221, 223, 225, 235, 237, 249}


<p>There are total 50 countries involving in this apple exporting trade. 28 countries exported apples to some of other countries while $50-28=22$ countries didn't as the apple providers</p>

In [ ]:
# only care about the quantities element=='Export Quantity'

In [10]:
# how many nodes on this graph
number_of_countries = data['Reporter Country Code']

In [ ]:
# get the ajacent matrix

In [74]:
import numpy as np
dic_ = {}
all_list = []
sparse = 0
for i in all_countries:
    one_row = []
    for j in all_countries:
        tuple_ = (i,j)
        query_result = only_quantity_data.loc[(only_quantity_data['Reporter Country Code']==i) & (only_quantity_data['Partner Country Code']==j)]['Value']
        query_result_list = query_result.to_list()
        # query doesn't get a result query_result_list == []
        try:
            # query_result_list has only one element
            dic_[tuple_] = query_result_list[0]
        except IndexError:
            if i == j:
                # get a degree diagonal matrix here 
                pass 
#                 dic_[tuple_] = degree
            dic_[tuple_] = 0 
        one_row.append(dic_[tuple_])
    one_row_array = np.array(one_row)
    all_list.append(one_row_array)
#             sparse = sparse + 1
#         print(tuple_, dic_[tuple_])

In [75]:
all_array = np.array(all_list)

In [76]:
all_array.shape

(50, 50)

In [79]:
df = pd.DataFrame(all_array, columns=all_countries, index=all_countries)

In [80]:
df.head()

,1,2,13,16,26,28,38,41,299,50,...,212,213,214,216,221,223,225,235,237,249
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
df.to_csv('apple_export_ajacent_matrix.csv')

In [72]:
sparse / len(dic_) 

0.9144

In [73]:
# visualize the ajacent matrix